In [1]:
#correct k-means

'''
Dynamic K-means clustering example in Jupyter Notebook.
'''

import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import display, HTML
import ipywidgets as widgets

def make_gaussians(cluster_n, img_size):
    """
    Generate Gaussian clusters for testing K-means.
    Returns points and their corresponding cluster labels.
    """
    points = []
    labels = []
    np.random.seed(42)

    for i in range(cluster_n):
        # Generate random cluster centers
        center_x, center_y = np.random.randint(0, img_size, 2)
        # Generate points around the cluster center
        sigma = img_size // (2 * cluster_n)
        pts = np.random.randn(100, 2) * sigma + [center_x, center_y]
        points.append(pts)
        labels.append(np.full(100, i))

    points = np.vstack(points).astype(np.float32)
    labels = np.hstack(labels)
    return points, labels

def dynamic_kmeans():
    cluster_n = 5
    img_size = 512

    # Generating bright palette
    colors = np.zeros((1, cluster_n, 3), np.uint8)
    colors[0, :] = 255
    colors[0, :, 0] = np.arange(0, 180, 180.0 / cluster_n)
    colors = cv.cvtColor(colors, cv.COLOR_HSV2BGR)[0]

    # Sampling data points
    points, _ = make_gaussians(cluster_n, img_size)

    # K-means criteria
    term_crit = (cv.TERM_CRITERIA_EPS, 30, 0.1)

    # Storage for each iteration's data
    all_labels = []
    all_centers = []

    # Perform the k-means algorithm step-by-step
    attempts = 10
    for i in range(attempts):
        _ret, labels, centers = cv.kmeans(points, cluster_n, None, term_crit, i + 1, 0)
        all_labels.append(labels.copy())
        all_centers.append(centers.copy())

    # Create the figure and axis
    fig, ax = plt.subplots(figsize=(8, 8))

    # Animation function
    def update(frame):
        ax.clear()
        ax.set_title(f'K-means Clustering - Iteration {frame + 1}')
        ax.axis('off')

        # Plot the points
        labels = all_labels[frame].ravel()
        centers = all_centers[frame]
        img = np.zeros((img_size, img_size, 3), np.uint8)

        for (x, y), label in zip(np.int32(points), labels):
            c = list(map(int, colors[label]))
            cv.circle(img, (x, y), 1, c, -1)

        # Plot the centers
        for center in centers:
            cv.circle(img, tuple(np.int32(center)), 5, (255, 255, 255), -1)

        ax.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))

    # Create animation
    ani = animation.FuncAnimation(fig, update, frames=len(all_labels), repeat=False, interval=500)

    # Display the animation
    plt.close(fig)
    return ani

# Button to generate a new distribution with dynamic animation
button = widgets.Button(description="Generate Dynamic K-means Clustering")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output(wait=True)
        ani = dynamic_kmeans()
        display(HTML(ani.to_jshtml()))  # Display animation as HTML

button.on_click(on_button_clicked)

# Display button and output in the notebook
display(button, output)


Button(description='Generate Dynamic K-means Clustering', style=ButtonStyle())

Output()